In [ ]:
# Don't change the names
!mkdir msdir
!mkdir input
!mkdir output

In [ ]:
import os
import json
import stimela
import pkg_resources
from drasticali import drasticali
from IPython.display import Javascript
from ipywidgets import widgets, interact
from drasticali.drasticali import natural_keys

In [ ]:
def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+4)'))

button = widgets.Button(description="Configure")
button.on_click(run_all)
display(button)

docker = widgets.Dropdown(options=['Docker', 'uDocker', 'Podman', 'Singularity'],
    value='uDocker',
    description='Container Tech'
)
display(docker)

from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit"
value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
DATA = dict()
MSDIR = 'msdir'
if docker.value in ['Docker']:
    recipe = stimela.Recipe('Make Pipeline', ms_dir=MSDIR, JOB_TYPE='docker')
elif docker.value in ['uDocker']:
    recipe = stimela.Recipe('Make Pipeline', ms_dir=MSDIR, JOB_TYPE='udocker')
elif docker.value in ['Podman']:
    recipe = stimela.Recipe('Make Pipeline', ms_dir=MSDIR, JOB_TYPE='podman')
elif docker.value in ['Singularity']:
    recipe = stimela.Recipe('Make Pipeline', ms_dir=MSDIR,
                            singularity_image_dir=os.environ["SINGULARITY_PULLFOLDER"])
cab_names = ['simms', 'simulator', 'calibrator', 'wsclean', 'ddfacet', 'cleanmask', 'casa_makemask',
             'casa_clean', 'lwimager', 'casa_tclean',
             'fitstool', 'pybdsm', 'aegean',]#stimela.CAB
def setup():
    for cab_name in cab_names:
        PARAM_FILE_PATH = pkg_resources.resource_filename('stimela', 'cargo/cab/{}'.format(cab_name))
        PARAM_FILE = '{}/parameters.json'.format(PARAM_FILE_PATH)

        with open(PARAM_FILE) as par:
            DATA[cab_name] = json.load(par)
setup()

In [ ]:
recipe_file = widgets.Text(
    value='',
    placeholder='recipe.json',
    description='Recipe File:',
    disabled=False
)
display(recipe_file)

In [ ]:
def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+3)'))

button = widgets.Button(description="LOAD")
button.on_click(run_all)
display(button)

In [ ]:
Im = []
recipe.jobs = []
data = drasticali.get_data(recipe_file.value)
cb_cont = widgets.HBox(background_color='red',height='200px',width='100%')
data_keys = data.keys()
data_keys.sort(key=natural_keys)
for dk in data_keys:
    Im.append(widgets.Checkbox(description=dk,value=False,width=50))
cb_cont.children=[i for i in Im]
display(cb_cont)

In [ ]:
def run_all(ev):
    # Runn all cells
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'
                       'IPython.notebook.get_selected_index()+5)'))

button = widgets.Button(description="RUN ALL")
button.on_click(run_all)
display(button)

In [ ]:
recipe.jobs = []
for cab_value in data_keys:
    cab_params = data[cab_value]
    new_cab_params = dict()
    cab_num = cab_value.split('_')[0]
    cab_name = cab_value.split('{}_'.format(cab_num))[-1]
    cab_func = getattr(drasticali, 'cab_function', None)
    for c in cb_cont.children:
        if c.description == cab_value:
            if c.value:
                for cab_param_key, cab_param_value in cab_params.items():
                    new_cab_params[str(cab_param_key)] = (str(cab_param_value)
                                                          if type(cab_param_value) is unicode
                                                          else cab_param_value)
                cab_func(recipe, name=cab_name, num=cab_num, parameters=new_cab_params,
                         recipes_file=recipe_file.value, dump=False)
drasticali.run_all(recipe, recipe_file.value)

## Execution Time

In [ ]:
from matplotlib import pyplot as plt
from aimfast import aimfast
import numpy as np
import math
%matplotlib inline

In [ ]:
cab_ids, times = [], []
prefix = recipe_file.value.split('/')[-1].split('.')[0]
data = drasticali.get_data('{}_time-it.json'.format(prefix))
data_keys = data.keys()
data_keys.sort(key=natural_keys)
for key in data_keys:
    #cab_id = key.split(prefix+'_')[-1]
    cab_id = key.split(prefix+'_')[-1].split('_')[0]
    time = float(data[key])
    cab_ids.append(cab_id)
    times.append(time)
aimfast.aimfast_plotly(cab_ids, np.array(times), plot_title='Time it', x_title='Cab', y_title='Time(m)', plot_mode1='markers+lines', yfactor=60)